In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\asus4\\OneDrive\\Documents\\DEEP LEARNING PROJECT\\DL-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\asus4\\OneDrive\\Documents\\DEEP LEARNING PROJECT\\DL-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True) 
class TrainingConfig: 
    root_dir: Path 
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epoch: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list  

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from src.cnnClassifier.utils.common import read_yaml, create_directories
from src.cnnClassifier.constants import *
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH):
    
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root]) 

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir), 
                            Path(config.tensorboard_log_dir)])
        
        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_log_dir=Path(config.tensorboard_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepare_callbacks_config
     
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data_path =os.path.join(self.config.data_ingestion.unzip_dir,"Chicken fickle images")
        create_directories([Path(training.root_dir)])
        
        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data_path),
            params_epoch=params.params_epoch,
            params_batch_size=params.params_batch_size,
            params_is_augmentation=params.params_is_augmentation,
            params_image_size=params.params_image_size
        )
        return training_config
        
        

In [8]:
import time

In [9]:
class prepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config      
    
    @property
    def _create_tensorboard_callback(self):
        tensorboard_log_dir = os.path.join(
            self.config.tensorboard_log_dir,
            f"tb_logs_at_{time.strftime('%Y-%m-%d-%H-%M-%S')}"
        )
        tensorboard_callback = tf.keras.callbacks.TensorBoard(
            log_dir=tensorboard_log_dir
        )
        return tensorboard_callback 
     
    @property
    def _create_model_checkpoint_callback(self):
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
        return model_checkpoint_callback
         
    @property
    def get_callbacks(self):
        return [
            self._create_tensorboard_callback,
            self._create_model_checkpoint_callback 
        ]

In [10]:
import os 
import urllib.request as request 
from zipfile import ZipFile 
import tensorflow as tf
tf.config.run_functions_eagerly(True)
print(f"Eager execution enabled: {tf.executing_eagerly()}")
import time

Eager execution enabled: True


In [14]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config 

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        self.model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy'])

    
    def train_valid_generator(self):
        datagen_kwargs = dict(rescale=1./255, validation_split=0.20)
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            class_mode="categorical"
        )
        valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagen_kwargs
        )
        self.valid_generator = valid_datagen.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        if self.config.params_is_augmentation:
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagen_kwargs
            )
        else:
            train_datagen = valid_datagen 
        
        self.train_generator = train_datagen.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    
    @staticmethod
    def save_model(model: tf.keras.Model, path: Path):
        model.save(path) 

    def train(self, callbacks_list: list):
        self.steps_per_epoch = int(self.train_generator.samples // self.config.params_batch_size) 
        self.validation_steps = int(self.valid_generator.samples // self.config.params_batch_size)
        
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epoch,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callbacks_list
        )
        self.save_model(
            model=self.model,
            path=self.config.trained_model_path
        )

In [15]:
try: 
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    
    prepare_callbacks = prepareCallbacks(config=prepare_callbacks_config)
    callbacks_list = prepare_callbacks.get_callbacks
    
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callbacks_list=callbacks_list)
except Exception as e:
    raise e

[2025-10-23 19:55:27,058]:INFO:common:yaml file: config\config.yaml loaded successfully]
[2025-10-23 19:55:27,067]:INFO:common:yaml file: params.yaml loaded successfully]
[2025-10-23 19:55:27,075]:INFO:common:created directory at : artifacts]


[2025-10-23 19:55:27,081]:INFO:common:created directory at : artifacts\prepare_callbacks\checkpoint_dir]
[2025-10-23 19:55:27,084]:INFO:common:created directory at : artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-10-23 19:55:27,093]:INFO:common:created directory at : artifacts\training]
[2025-10-23 19:55:27,645]:WARNING:saving_utils:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.


c:\Users\asus4\OneDrive\Documents\DEEP LEARNING PROJECT\DL-Project\chicken\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
c:\Users\asus4\OneDrive\Documents\DEEP LEARNING PROJECT\DL-Project\chicken\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6371 - loss: 6.2260[2025-10-23 19:56:44,115]:WARNING:saving_api:You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
19/19 ━━━━━━━━━━━━━━━━━━━━ 76s 4s/step - accuracy: 0.6757 - loss: 5.5765 - val_accuracy: 0.8906 - val_loss: 2.5138
[2025-10-23 19:56:44,250]:WARNING:saving_api:You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
